This notebook contains study notes of the short course [ChatGPT Prompt Engineering for Developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers) of DeepLearning.AI.

In [1]:
#| echo: false
import sys
sys.path.append('../src')

In [2]:
#| code-fold: true
#| code-summary: Import libs and functions (click to toggle the content)
import json
import openai

from openai_utils import chat_completion

In [3]:
#| code-fold: true
#| code-summary: Enter OpenAI API Key (click to toggle the content)
from getpass import getpass

openai.api_key = getpass()

 ········


## How to write clear and specific  instructions
### Use delimiters

In [4]:
text = f"""\
You should express what you want a model to do by providing instructions that are as clear and specific as you can possibly make them. This will guide the model towards the desired output, and reduce the chances of receiving irrelevant or incorrect responses. Don't confuse writing a clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity and context for the model, which can lead to more detailed and relevant outputs."""
prompt = f"""\
Summarize the text delimited by triple backticks into a single sentence.
```{text}```
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, _ = chat_completion(messages)
print(response)

To guide a model towards the desired output and minimize irrelevant or incorrect responses, it is important to provide clear and specific instructions, even if they are longer and more detailed.


### Ask for structured outputs

In [5]:
prompt = f"""\
Generate a list of three made-up book titles along with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, _ = chat_completion(messages)
print(response)

{
  "books": [
    {
      "book_id": 1,
      "title": "The Enigma of Elysium",
      "author": "Evelyn Sinclair",
      "genre": "Mystery"
    },
    {
      "book_id": 2,
      "title": "Whispers in the Wind",
      "author": "Lucas Montgomery",
      "genre": "Fantasy"
    },
    {
      "book_id": 3,
      "title": "Shadows of Serendipity",
      "author": "Amelia Hart",
      "genre": "Romance"
    }
  ]
}


### Specify conditions in prompts

In [6]:
user_input = f"""\
Making a cup of tea is easy! First, you need to get some water boiling. While that's happening, grab a cup and put a tea bag in it. Once the water is hot enough, just pour it over the tea bag. Let it sit for a bit so the tea can steep. After a few minutes, take out the tea bag. If you like, you can add some sugar or milk to taste. And that's it! You've got yourself a delicious cup of tea to enjoy."""

prompt = f"""\
You will be provided with text delimited by triple backticks.
If it contains a sequence of instructions, re-write those instructions in the following format:

Step 1 - ...
Step 2 - ...
…
Step N - ...

If the text does not contain a sequence of instructions, then simply write "No steps provided."

```{user_input}```
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, _ = chat_completion(messages)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your cup of tea.


In [7]:
user_input = f"""\
The sun is shining brightly today, and the birds are singing. It's a beautiful day to go for a walk in the park. The flowers are blooming, and the trees are swaying gently in the breeze. People are out and about, enjoying the lovely weather. Some are having picnics, while others are playing games or simply relaxing on the grass. It's a perfect day to spend time outdoors and appreciate the beauty of nature."""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, re-write those instructions in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, then simply write "No steps provided."

\"\"\"{user_input}\"\"\"
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, _ = chat_completion(messages)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


### Few-shot prompting

In [8]:
prompt = f"""\
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley flows from a modest spring; the grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, _ = chat_completion(messages)
print(response)

<grandparent>: Resilience is like a mighty oak tree that withstands the strongest storms, bending but never breaking. It is the unwavering spirit that rises from the ashes, stronger and more determined than before.


## Let the model "think"
### Specify steps for completing a task

In [9]:
text = f"""\
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled down the hill, with Jill following suit. Though slightly battered, the pair returned home to comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they continued exploring with delight."""

prompt_1 = f"""\
Perform the following actions: 
1 - Summarize the following text delimited by triple backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
messages = [
    {'role': 'user', 'content': prompt_1}
]
response, _ = chat_completion(messages)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill, siblings, go on a quest to fetch water from a well on a hill, but they both fall down the hill after Jack trips on a stone, yet they return home and remain adventurous.
2 - Jack et Jill, frère et sœur, partent en quête d'eau d'un puits situé au sommet d'une colline, mais ils tombent tous les deux après que Jack trébuche sur une pierre, cependant ils rentrent chez eux et restent aventureux.
3 - Jack, Jill
4 - {"french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits situé au sommet d'une colline, mais ils tombent tous les deux après que Jack trébuche sur une pierre, cependant ils rentrent chez eux et restent aventureux.", "num_names": 2}


#### specify output format

In [11]:
prompt_2 = f"""\
Your task is to perform the following actions:
1 - Summarize the following text delimited by <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in French summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
messages = [
    {'role': 'user', 'content': prompt_2}
]
response, _ = chat_completion(messages)
print("\nCompletion for prompt 2:")
print(response)

Completion for prompt 2:
Summary: Jack and Jill, siblings from a charming village, go on a quest to fetch water from a hilltop well but encounter misfortune along the way. 
Translation: Jack et Jill, frère et sœur d'un charmant village, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill, frère et sœur d'un charmant village, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.", "num_names": 2}


### Ask the model to work out its own solution before reaching a conclusion

In [12]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot

What is the total cost for the first year of operations as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, _ = chat_completion(messages)
print(response)

The student's solution is correct. The total cost for the first year of operations is indeed 450x + 100,000.


The model's response is wrong because the student's solution is not correct. To fix it, ask the model to figure out its own solution.

In [13]:
prompt = f"""
Your task is to determine if the student's solution is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot

What is the total cost for the first year of operations as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

messages = [
    {'role': 'user', 'content': prompt}
]
response, _ = chat_completion(messages)
print(response)

To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

1. Land cost: $100 / square foot
The cost of land is $100 multiplied by the number of square feet.

2. Solar panel cost: $250 / square foot
The cost of solar panels is $250 multiplied by the number of square feet.

3. Maintenance cost: $100,000 + $10 / square foot
The maintenance cost is a flat fee of $100,000 per year, plus $10 multiplied by the number of square feet.

Total cost = Land cost + Solar panel cost + Maintenance cost

Total cost = ($100 / square foot) * x + ($250 / square foot) * x + ($100,000 + $10 / square foot) * x

Simplifying the expression:

Total cost = $100x + $250x + ($100,000 + $10x)

Total cost = $350x + $100,000

Is the student's solution the same as the actual solution just calculated:
No


## An example of iterative prompt development

In [15]:
#| code-fold: true
#| code-summary: product fact sheet (click to toggle the content)

fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [21]:
#| code-fold: true
#| code-summary: a helper function (click to toggle the content)

def run_and_print(prompt):
    messages = [
        {'role': 'user', 'content': prompt}
    ]
    response, _ = chat_completion(messages)
    print(response)
    
    return response

In [16]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""

run_and_print(prompt)

Introducing our stunning mid-century inspired office chair, the perfect addition to any home or business setting. This chair is part of a beautiful family of office furniture, including filing cabinets, desks, bookcases, meeting tables, and more, all designed with a timeless mid-century aesthetic.

One of the standout features of this chair is the variety of customization options available. You can choose from several shell colors and base finishes to perfectly match your existing decor. The chair is available with either plastic back and front upholstery or full upholstery in a range of 10 fabric and 6 leather options, allowing you to create a look that is uniquely yours.

The chair is also available with or without armrests, giving you the flexibility to choose the option that best suits your needs. The base finish options include stainless steel, matte black, gloss white, or chrome, ensuring that you can find the perfect match for your space.

In terms of construction, this chair is

### The text is too long, limit the length of response

In [18]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

run_and_print(prompt)

Introducing our mid-century inspired office chair, part of a stunning furniture collection. With various color and finish options, choose between plastic or full upholstery in fabric or leather. The chair features a durable aluminum base with 5 wheels and pneumatic height adjustment. Perfect for home or business use. Made in Italy.


### Try to ask the model to focus on aspects for intended audience

In [19]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

run_and_print(prompt)

Introducing our mid-century inspired office chair, part of a beautiful furniture collection. With various shell colors and base finishes, it offers versatility for any setting. Choose between plastic or full upholstery in a range of fabric and leather options. The chair features a durable aluminum base with 5-wheel design and pneumatic chair adjustment. Made in Italy.


In [20]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

run_and_print(prompt)

Introducing our mid-century inspired office chair, part of a beautiful family of furniture. With various shell colors and base finishes, this chair offers versatility and style. Choose between plastic or full upholstery in a range of fabric and leather options. The chair features a 5-wheel plastic coated aluminum base and a pneumatic chair adjust for easy height adjustment. Available with or without armrests, this chair is suitable for both home and business settings. Made with high-quality materials, including a cast aluminum shell and HD36 foam seat, this chair is built to last. Product ID: SWC-100, SWC-110.


### Ask the model to respond using HTML format with a table

In [23]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

After the description, include a table that gives the 
product's dimensions. The table should have two columns. 
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.
Give the table the title "Product Dimensions".

After the table, list all 7-character Product IDs from the technical specification.

Technical specifications: ```{fact_sheet_chair}```

Format everything as HTML that can be used in a website. 
Place the table after the description.
"""

messages = [
    {'role': 'user', 'content': prompt}
]
response, _ = chat_completion(messages, max_tokens=800)
print(response)

<!DOCTYPE html>
<html>
<head>
  <title>Product Description</title>
</head>
<body>
  <h1>Product Description</h1>
  <p>Introducing our latest addition to the mid-century inspired office furniture collection - the SWC Chair. This beautifully designed chair is perfect for both home and business settings, offering comfort and style in one package. With its sleek and modern look, it is sure to enhance any workspace.</p>
  <p>The SWC Chair is constructed with high-quality materials to ensure durability and longevity. The shell is made of cast aluminum with a modified nylon PA6/PA66 coating, providing strength and stability. The seat is filled with HD36 foam, offering exceptional comfort for extended periods of sitting.</p>
  <p>With a 5-wheel plastic coated aluminum base, the SWC Chair provides smooth mobility and stability. The pneumatic chair adjustment allows for easy raise and lower action, ensuring the perfect height for any user.</p>
  
  <h2>Product Dimensions</h2>
  <table>
    <tr>


In [24]:
from IPython.display import display, HTML

display(HTML(response))

Dimension,Measurement (inches)
Width,"20.87"""
Depth,"20.08"""
Height,"31.50"""
Seat Height,"17.32"""
Seat Depth,"16.14"""
